This first part only looks for the papers for which the DOI has been indicated in the assignments csv file. It produces an output called "useful1.csv" which would eventually later be used by a second part of the program. 

In [1]:
from webbot import Browser
import pandas as pd
import time
import glob
import os
import tqdm
import re

In [13]:
site = Browser(downloadPath='')
df = pd.read_csv('assignments.csv', index_col = 0,dtype = str)

In [3]:
righe = df.index
col = ['Handle', 'Anno di pubblicazione', 'DOI', 'scopus: Identificativo']

sub_df = df.loc[righe, col]
#non null items
new_df = sub_df.dropna(axis = 0, how = 'any')

new_df
#obtaining useful dois
dois = new_df['DOI']

title = []
righe = new_df.index
for riga in righe:
    title.append('VE_'+new_df.loc[riga, 'Anno di pubblicazione']
          +'_'+new_df.loc[riga, 'Handle'][:5]+"-"+new_df.loc[riga,'Handle'][6:])
new_df.insert(loc = 4, column = 'Titoli', value = title)

In [ ]:
site = Browser(downloadPath='step' #is a folder
              )
url_proxy="https://login.ezproxy.uniroma1.it/login"
url_scopus="https://www.scopus.com/search/form.uri?display=basic#basic"

#accessing via proxy to...
#site.go_to(url_proxy)
#site.type('myemail', into='email')
#site.type('mypassword',into='*********')
#site.click('Invia')

#...scopus
site.go_to(url_scopus)

#avoiding annoying blocks
site.click('Continue as guest')

for riga in new_df.index:
    print('Working on row *** {} *** '.format(riga))
    
    site.click('DOI')
    site.click('Search documents')
    site.click('Search documents')
    site.type(str(new_df.loc[riga, 'DOI']))
    site.click('Search','button','DocumentSearchForm__submitButton___16LlX')
    site.click(classname='ddmDocTitle')
    
    site.click('View at Publisher')
    
    time.sleep(.2)
    
    site.go_to(url_scopus)
    site.click('DOI')
    site.click('Search documents')
    
    #what to automate with ML so that it completely works without any, even minimal, supervision
    input('When ready for next iteration press enter: ')

In [4]:
ddf = pd.read_csv('new_assigments.csv')
new_df_2 = new_df.copy()
new_df_2 = new_df_2.drop(axis = 0,
             #dropping non useful rows
             labels = [
                 5,14,32,36,44,51,59,62,66,69,73,77,90,92,98,103,113,114,116,119,130,131,132,134,138,153,154,155,
                 157,163,166,171,173,174,185,191,196,198
             ])
for indice in new_df_2.index:
    #print(indice)
    pass

In [ ]:
#obtaing the desiderate rename process
for index in tqdm.tqdm(new_df_2.index):
    os.rename('step/'+str(index)+".pdf",
              'remote/'+new_df_2.at[index, 'Titoli']+'.pdf')

In [ ]:
#obtaining a .csv to sum up the results of the iterations

labels = [5,14,32,36,44,51,59,62,66,69,73,77,90,92,98,103,113,114,116,
          119,130,131,132,134,138,153,154,155,157,163,166,171,173,174,185,191,196,198
         ]

for indice in df.index:
    df.at[indice, 'note'] = ''
    if indice in new_df_2.index:
        df.at[indice, 'fatto'] = 'x'
    
    if indice in labels:
        df.at[indice, 'note'] = 'pdf non reperibile (Springer, Bad link o DOI errato)'
        
    if indice not in new_df.index:
        df.at[indice, 'note'] = 'DOI non presente'   

df.to_csv(path_or_buf='useful1.csv')